# RAG FAQ-app
 

### Step 1: Load the Data

In [12]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader('dockuno.csv')
dataset = loader.load()

### Step 2: Load embeddings

In [19]:
# from langchain.embeddings.openai import OpenAIEmbeddings
# myembedmodels = OpenAIEmbeddings(openai_api_key="")

from langchain_community.embeddings import OllamaEmbeddings
myembedmodels = OllamaEmbeddings(model="all-minilm")

C:\Users\saura\AppData\Local\Temp\ipykernel_18340\4073483054.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  myembedmodels = OllamaEmbeddings(model="all-minilm")


### Step 3: store embeddings into Vector Database

In [22]:
# pip install -qU langchain-community faiss-cpu
from langchain.vectorstores import FAISS
db = FAISS.from_documents( documents=dataset , embedding=myembedmodels)

In [25]:
db.similarity_search("What is the fees?")

[Document(id='a93e060c-b70f-4779-a953-9300ab42a0d6', metadata={'source': 'dockuno.csv', 'row': 1}, page_content='Questions: What is the fee for the course?\nAnswers: The course fee depends on the selected package.\nPlease visit our official website for detailed pricing, including discounts and offers.'),
 Document(id='9abc0ed8-ec50-49ff-9439-8ac0a9e76eb1', metadata={'source': 'dockuno.csv', 'row': 18}, page_content='Questions: Are live Q&A sessions included in the course?\nAnswers: Yes, live Q&A sessions are held periodically where you can ask questions directly to instructors.\nThese sessions help clarify doubts and provide additional insights.'),
 Document(id='02f9f9ec-1c66-4926-af42-dc567d9c780c', metadata={'source': 'dockuno.csv', 'row': 15}, page_content='Questions: Are there any group discounts available?\nAnswers: Yes, we offer group discounts for corporate teams and student groups.\nContact us for more information on bulk enrollment discounts.'),
 Document(id='c6176c22-226c-4d9